In [25]:
import netCDF4 as nc

nncamrh_path = "/Users/jianda/Projects/experience_replay_data/nncamrh/nc_files/baseline_nn_rh.cam.h1.0001-01-01-00000.nc"
nncamrh = nc.Dataset(nncamrh_path)
print(nncamrh)

<class 'netCDF4._netCDF4.Dataset'>
root group (NETCDF3_64BIT_OFFSET data model, file format NETCDF3):
    Conventions: CF-1.0
    source: CAM
    case: baseline_nn_rh
    title: UNSET
    logname: chenj209
    host: ai002
    Version: $Name$
    revision_Id: $Id$
    initial_file: /share2/CESM/cesm-input//atm/cam/inic/fv/cami-mam3_0000-01-01_1.9x2.5_L30_c090306.nc
    topography_file: /share2/CESM/cesm-input//atm/cam/topo/USGS-gtopo30_1.9x2.5_remap_c050602.nc
    dimensions(sizes): lat(96), lon(144), slat(95), slon(144), time(8760), nbnd(2), chars(8), lev(30), ilev(31), crm_nx(32), crm_ny(1), crm_nz(28), pverp(31), pver(30), NCLASS_CL(2), ncls_ecpp_in(3), NCLASS_PR(2)
    variables(dimensions): float64 lev(lev), float64 hyam(lev), float64 hybm(lev), float64 ilev(ilev), float64 hyai(ilev), float64 hybi(ilev), float64 P0(), float64 time(time), int32 date(time), int32 datesec(time), float64 lat(lat), float64 lon(lon), float64 slat(slat), float64 slon(slon), float64 w_stag(slat), float64 t

In [26]:
print(nncamrh.variables["time"])
print(nncamrh.variables["time"][:10])

<class 'netCDF4._netCDF4.Variable'>
float64 time(time)
    long_name: time
    units: days since 0001-01-01 00:00:00
    calendar: noleap
    bounds: time_bnds
unlimited dimensions: time
current shape = (8760,)
filling on, default _FillValue of 9.969209968386869e+36 used
[0.         0.04166667 0.08333333 0.125      0.16666667 0.20833333
 0.25       0.29166667 0.33333333 0.375     ]


In [28]:
print(nncamrh.variables["date"])
print(nncamrh.variables["date"][:10])

<class 'netCDF4._netCDF4.Variable'>
int32 date(time)
    long_name: current date (YYYYMMDD)
unlimited dimensions: time
current shape = (8760,)
filling on, default _FillValue of -2147483647 used
[10101 10101 10101 10101 10101 10101 10101 10101 10101 10101]


In [29]:
var_name = "time"
print(nncamrh.variables[var_name])
print(nncamrh.variables[var_name][:10])
var_name = "time_bnds"
print(nncamrh.variables[var_name])
print(nncamrh.variables[var_name][:10])
var_name = "date"
print(nncamrh.variables[var_name])
print(nncamrh.variables[var_name][:10])


<class 'netCDF4._netCDF4.Variable'>
float64 time(time)
    long_name: time
    units: days since 0001-01-01 00:00:00
    calendar: noleap
    bounds: time_bnds
unlimited dimensions: time
current shape = (8760,)
filling on, default _FillValue of 9.969209968386869e+36 used
[0.         0.04166667 0.08333333 0.125      0.16666667 0.20833333
 0.25       0.29166667 0.33333333 0.375     ]
<class 'netCDF4._netCDF4.Variable'>
float64 time_bnds(time, nbnd)
    long_name: time interval endpoints
unlimited dimensions: time
current shape = (8760, 2)
filling on, default _FillValue of 9.969209968386869e+36 used
[[0.         0.        ]
 [0.         0.04166667]
 [0.04166667 0.08333333]
 [0.08333333 0.125     ]
 [0.125      0.16666667]
 [0.16666667 0.20833333]
 [0.20833333 0.25      ]
 [0.25       0.29166667]
 [0.29166667 0.33333333]
 [0.33333333 0.375     ]]
<class 'netCDF4._netCDF4.Variable'>
int32 date(time)
    long_name: current date (YYYYMMDD)
unlimited dimensions: time
current shape = (8760,)
fi

In [31]:
from netCDF4 import num2date
import numpy as np
from datetime import datetime, timedelta
# Create a new netCDF file
output_path = "./baseline_nn_rh_1998.nc"
new_nc = nc.Dataset(output_path, 'w')

# Create a function to convert time values to YYYYMMDD format
def time_to_date(time_values, units, calendar='noleap'):
    # Convert time values to datetime objects
    dates = num2date(time_values, units, calendar=calendar)
    # Convert to YYYYMMDD format
    date_ints = np.array([int(d.strftime('%Y%m%d')) for d in dates])
    return date_ints

# Copy all dimensions from the original file
for dim_name, dim in nncamrh.dimensions.items():
    new_nc.createDimension(dim_name, len(dim) if not dim.isunlimited() else None)

# When creating the new netCDF file, modify the date variable
for var_name, var in nncamrh.variables.items():
    new_var = new_nc.createVariable(var_name, var.datatype, var.dimensions)
    
    if var_name == 'date':
        # Compute new dates based on the time variable
        time_values = nncamrh.variables['time'][:]
        new_dates = time_to_date(time_values, 'days since 1998-01-01 00:00:00', calendar='noleap')
        new_var[:] = new_dates
    else:
        # Copy data for all other variables
        new_var[:] = var[:]
    
    # Copy attributes
    for attr_name in var.ncattrs():
        if var_name == 'time' and attr_name == 'units':
            new_var.setncattr('units', 'days since 1998-01-01 00:00:00')
        else:
            new_var.setncattr(attr_name, var.getncattr(attr_name))

# Copy global attributes
for attr_name in nncamrh.ncattrs():
    new_nc.setncattr(attr_name, nncamrh.getncattr(attr_name))

# Close both files
new_nc.close()
nncamrh.close()

# Verify the changes (optional)
check = nc.Dataset(output_path)
print("New time variable attributes:")
print(check.variables['time'])
print("\nFirst 10 time values:")
print(check.variables['time'][:10])
check.close()

New time variable attributes:
<class 'netCDF4._netCDF4.Variable'>
float64 time(time)
    long_name: time
    units: days since 1998-01-01 00:00:00
    calendar: noleap
    bounds: time_bnds
unlimited dimensions: time
current shape = (8760,)
filling on, default _FillValue of 9.969209968386869e+36 used

First 10 time values:
[0.         0.04166667 0.08333333 0.125      0.16666667 0.20833333
 0.25       0.29166667 0.33333333 0.375     ]


In [32]:
check = nc.Dataset(output_path)
print(check.variables["time"])
print(check.variables["time"][:10])
print(check.variables["date"])
print(check.variables["date"][:10])
check.close()

<class 'netCDF4._netCDF4.Variable'>
float64 time(time)
    long_name: time
    units: days since 1998-01-01 00:00:00
    calendar: noleap
    bounds: time_bnds
unlimited dimensions: time
current shape = (8760,)
filling on, default _FillValue of 9.969209968386869e+36 used
[0.         0.04166667 0.08333333 0.125      0.16666667 0.20833333
 0.25       0.29166667 0.33333333 0.375     ]
<class 'netCDF4._netCDF4.Variable'>
int32 date(time)
    long_name: current date (YYYYMMDD)
unlimited dimensions: time
current shape = (8760,)
filling on, default _FillValue of -2147483647 used
[19980101 19980101 19980101 19980101 19980101 19980101 19980101 19980101
 19980101 19980101]


In [34]:
def convert_to_1998_dates(input_filename):
    """
    Convert a netCDF file's time and date variables to use 1998 as the reference year.
    
    Args:
        input_filename (str): Path to input netCDF file
    
    Returns:
        str: Path to the output file
    """
    # Open input file
    input_nc = nc.Dataset(input_filename)
    
    # Create output filename by adding _1998 before the .nc extension
    # output_filename = input_filename.rsplit('.nc', 1)[0] + '_1998.nc'
    output_filename = input_filename.replace('0001', '1998')
    new_nc = nc.Dataset(output_filename, 'w')
    
    # Create a function to convert time values to YYYYMMDD format
    def time_to_date(time_values, units, calendar='noleap'):
        dates = num2date(time_values, units, calendar=calendar)
        date_ints = np.array([int(d.strftime('%Y%m%d')) for d in dates])
        return date_ints
    
    # Copy dimensions
    for dim_name, dim in input_nc.dimensions.items():
        new_nc.createDimension(dim_name, len(dim) if not dim.isunlimited() else None)
    
    # Copy variables with modified time/date
    for var_name, var in input_nc.variables.items():
        new_var = new_nc.createVariable(var_name, var.datatype, var.dimensions)
        
        if var_name == 'date':
            # Compute new dates based on the time variable
            time_values = input_nc.variables['time'][:]
            new_dates = time_to_date(time_values, 'days since 1998-01-01 00:00:00', calendar='noleap')
            new_var[:] = new_dates
        else:
            # Copy data for all other variables
            new_var[:] = var[:]
        
        # Copy attributes
        for attr_name in var.ncattrs():
            if var_name == 'time' and attr_name == 'units':
                new_var.setncattr('units', 'days since 1998-01-01 00:00:00')
            else:
                new_var.setncattr(attr_name, var.getncattr(attr_name))
    
    # Copy global attributes
    for attr_name in input_nc.ncattrs():
        new_nc.setncattr(attr_name, input_nc.getncattr(attr_name))
    
    # Close files
    new_nc.close()
    input_nc.close()
    
    return output_filename


In [35]:
convert_to_1998_dates(nncamrh_path)

'/Users/jianda/Projects/experience_replay_data/nncamrh/nc_files/baseline_nn_rh.cam.h1.1998-01-01-00000.nc'

In [36]:
original_file = nncamrh_path
check_file = nncamrh_path.replace('0001', '1998')
check = nc.Dataset(check_file)
print(check.variables["time"])
print(check.variables["time"][:10])
print(check.variables["date"])
print(check.variables["date"][:10])
original_file = nc.Dataset(original_file)
print(original_file.variables["time"])
print(original_file.variables["time"][:10])
print(original_file.variables["date"])
print(original_file.variables["date"][:10])

<class 'netCDF4._netCDF4.Variable'>
float64 time(time)
    long_name: time
    units: days since 1998-01-01 00:00:00
    calendar: noleap
    bounds: time_bnds
unlimited dimensions: time
current shape = (8760,)
filling on, default _FillValue of 9.969209968386869e+36 used
[0.         0.04166667 0.08333333 0.125      0.16666667 0.20833333
 0.25       0.29166667 0.33333333 0.375     ]
<class 'netCDF4._netCDF4.Variable'>
int32 date(time)
    long_name: current date (YYYYMMDD)
unlimited dimensions: time
current shape = (8760,)
filling on, default _FillValue of -2147483647 used
[19980101 19980101 19980101 19980101 19980101 19980101 19980101 19980101
 19980101 19980101]
<class 'netCDF4._netCDF4.Variable'>
float64 time(time)
    long_name: time
    units: days since 0001-01-01 00:00:00
    calendar: noleap
    bounds: time_bnds
unlimited dimensions: time
current shape = (8760,)
filling on, default _FillValue of 9.969209968386869e+36 used
[0.         0.04166667 0.08333333 0.125      0.16666667 

In [37]:
np.allclose(check.variables["cp"][:], original_file.variables["cp"][:])

True

In [ ]:
np.allclose(check.variables["rh"][:], original_file.variables["rh"][:])

In [40]:
for var in original_file.variables:
    print(var)
    try:
        print(np.allclose(check.variables[var][:], original_file.variables[var][:]))
    except:
        print(var + " is not checked")

lev
True
hyam
True
hybm
True
ilev
True
hyai
True
hybi
True
P0
True
time
True
date
False
datesec
True
lat
True
lon
True
slat
True
slon
True
w_stag
True
time_bnds
True
date_written
date_written is not checked
time_written
time_written is not checked
ntrm
True
ntrn
True
ntrk
True
ndbase
True
nsbase
True
nbdate
True
nbsec
True
mdt
True
nlon
True
wnummax
True
gw
True
ndcur
True
nscur
True
co2vmr
True
ch4vmr
True
n2ovmr
True
f11vmr
True
f12vmr
True
sol_tsi
True
nsteph
True
FLUT
True
OMEGA500
True
U200
True
U850
True
cp
True
